# Advanced Analysis & Statistics
- **Comparative Velocity Modeling:** Visualize the kinetic energy difference between natural meteoroids (~20 km/s) and man-made orbital debris (~7.8 km/s).
- **Density Threshold Mapping:** Create a "Kessler Limit" overlay to show exactly where man-made density exceeds the background natural environment.
- **Risk Attribution:** Quantify the probability of a "Mission Ending Strike" from a 1cm man-made fragment versus a natural micrometeoroid of the same size.
- **Predictive Tracking Decay:** Use Correlation Matrices to determine if object age is a leading indicator for "unhealthy" tracking status.
- **Physical Risk Modeling:** Utilize RCS (Radar Cross Section) data to calculate the specific collision probability of "Zombie Satellites" versus standard debris fragments.

# **Future Integration: Geospatial Analysis with GeoPandas**
While the current analysis focuses on altitude and mass, the next logical step is to project these orbits onto Earth's surface. 

**Objectives:**
* Use `GeoPandas` to create a heatmap of satellite "density" over specific geographic regions.
* Map the launch sites (from the UCS data) to see if launch location correlates with specific orbital congestion zones.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# --- PHYSICS CONSTANTS ---
# We need these to convert "Altitude" into "Velocity" (Vis-viva equation)
MU = 398600.4418   # Standard gravitational parameter (km^3/s^2)
R_EARTH = 6378.137 # Earth's equatorial radius (km)

clutter_data = pd.read_csv('../data/clean/orbital_clutter_cleaned.csv')

print(f"✅ Physics Engine Loaded.")
print(f"   - Objects: {len(clutter_data):,}")
print(f"   - Mass Data: {clutter_data['proxy_mass_kg'].notnull().sum() / len(clutter_data):.1%} coverage (Tier 1)")
print(f"   - RCS Data: {clutter_data['rcs'].notnull().sum() / len(clutter_data):.1%} coverage (Tier 2)")

✅ Physics Engine Loaded.
   - Objects: 32,695
   - Mass Data: 99.9% coverage (Tier 1)
   - RCS Data: 45.3% coverage (Tier 2)


In [4]:
# --- TIER 2 PHYSICS ENGINE ---
# Calculating Orbital Velocity and Kinetic Energy (Joules)

def calculate_orbital_physics(df):
    """
    Applies the Vis-viva equation to calculate velocity and kinetic energy.
    """
    df = df.copy()
    
    # 1. Calculate Orbital Radius (Distance from Earth's Center)
    # r = Earth Radius + Altitude
    df['radius_km'] = R_EARTH + df['perigee_km']
    
    # 2. Calculate Velocity (Vis-viva Equation for Circular Orbit)
    # v = sqrt(mu / r)
    df['velocity_kms'] = np.sqrt(MU / df['radius_km'])
    
    # 3. Calculate Kinetic Energy (Joules)
    # KE = 1/2 * m * v^2
    # CRITICAL: Velocity must be in METERS/second for Joules.
    # We multiply velocity_kms by 1000 to get m/s.
    df['kinetic_joules'] = 0.5 * df['proxy_mass_kg'] * (df['velocity_kms'] * 1000)**2
    
    return df

# Apply the Physics
clutter_data = calculate_orbital_physics(clutter_data)

# --- VERIFICATION ---
# 1. Print columns to be sure of the names
print("Available Columns:", clutter_data.columns.tolist())

# 2. Define the columns to display (using the likely correct names)
# We use 'category' because your Traffic Map used it.
# We use 'object_name' as the standard default.
cols = ['object_name', 'category', 'perigee_km', 'velocity_kms', 'proxy_mass_kg', 'kinetic_joules']

# 3. Show the "Top Guns"
print("\n--- TOP 10 HIGHEST ENERGY OBJECTS ---")
display(clutter_data[cols].sort_values('kinetic_joules', ascending=False).head(10))

Available Columns: ['object_name', 'object_id', 'norad_id', 'object_type', 'ops_status', 'owner', 'launch_date', 'launch_site', 'decay_date', 'period_minutes', 'inclination_degrees', 'apogee_km', 'perigee_km', 'rcs', 'data_status', 'orbit_center', 'orbit_type', 'orbit_class', 'users', 'purpose', 'launch_mass_kg', 'source', 'launch_year', 'decay_year', 'category', 'proxy_mass_kg', 'radius_km', 'velocity_kms', 'kinetic_joules']

--- TOP 10 HIGHEST ENERGY OBJECTS ---


,object_name,category,perigee_km,velocity_kms,proxy_mass_kg,kinetic_joules
6848,ISS (ZARYA),Active Satellite,414.0,7.660651,450000.0,1.320425e+13
19486,USA 326,Active Satellite,497.0,7.614269,20000.0,5.797709e+11
18366,USA 314,Active Satellite,550.0,7.585089,20000.0,5.753357e+11
12400,USA 224,Active Satellite,258.0,7.750170,18000.0,5.405862e+11
8340,USA 186,Active Satellite,264.0,7.746669,18000.0,5.400979e+11
13504,USA 245,Active Satellite,268.0,7.744337,18000.0,5.397728e+11
16375,USA 290,Active Satellite,400.0,7.668558,18000.0,5.292611e+11
8252,USA 182,Active Satellite,715.0,7.496348,14500.0,4.074154e+11
5027,HST,Active Satellite,479.0,7.624256,11110.0,3.229082e+11
13472,COSMOS 2486,Active Satellite,698.0,7.505347,7000.0,1.971558e+11
